In [3]:
import os
import pickle
import time

from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# OAuth 2.0 credentials
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = r"D:\OneDrive\OneDrive - Rowad Modern Engineering\x004 Data Science\03.rme.db\00.repo\rme.db\09.api.projects\01.youtube\02.upload.with.api\client_secret_253406904901-shu0pjqhddobn21msfjn1o7ue5q3k020.apps.googleusercontent.com.json"  # Replace with the path to your downloaded JSON file

def get_authenticated_service():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('youtube', 'v3', credentials=creds)



def upload_to_youtube(video_path, title, description, tags=[], privacyStatus='unlisted'):
    """
    Uploads a video to YouTube with a progress indicator.

    Args:
        video_path (str): Path to the video file.
        title (str): Title of the video.
        description (str): Description of the video.
        tags (list, optional): List of tags for the video. Defaults to [].
        privacyStatus (str, optional): Privacy status of the video ('private', 'public', 'unlisted'). Defaults to 'unlisted'.
    """

    youtube = get_authenticated_service()

    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": title,
                "description": description,
                "tags": tags,
                "madeForKids": False
            },
            "status": {
                "privacyStatus": privacyStatus
            }
        },
        media_body=MediaFileUpload(video_path, chunksize=-1, resumable=True)
    )

    response = None
    while response is None:
        try:
            status, response = request.next_chunk()
            if status:
                # Clear the previous line and print the updated progress
                print(" " * 80, end="\r")  # Clear the line
                print(f"Uploaded {int(status.progress() * 100)}%", end="\r")
                time.sleep(0.1)  # Add a small delay for better visual feedback
        except:  # Handle potential errors during upload
            print("An error occurred during upload.")
            return

    print(f"Video uploaded successfully! Video ID: {response['id']}")


def upload_videos_from_folder(folder_path):
    """
    Uploads all videos from a folder to YouTube.

    Args:
        folder_path (str): Path to the folder containing the videos.
    """

    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            video_path = os.path.join(folder_path, filename)
            title = filename[:-4]  # Remove the .mp4 extension from the filename
            description = f"Uploaded from folder: {folder_path}"
            upload_to_youtube(video_path, title, description)

# Get the folder path
folder_path = input("Enter the folder path containing the videos: ")

# Upload videos from the folder
upload_videos_from_folder(folder_path)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=253406904901-shu0pjqhddobn21msfjn1o7ue5q3k020.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56892%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload&state=88slHzMvqrU9vmoOR7AHgP6LuqnSGj&access_type=offline


SSLError: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))